In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pipreqs

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 3.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=1f8693de55d38276f2336d0aed1566bbfa3a56b629334080989f45a43d3b7396
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Attempting uninstall: mistune
    Found existing installation: mistune 0.8.4
    Uninstalling mistune-0.8.4:
      Successfully uninstalled mistune-0.8.4
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
  Attempting uninstall: nbconvert
    Fo

In [8]:
!pipreqs "/content/drive/MyDrive/Dicoding/ProjectAnalisisSentimen/Script/Done/" --scan-notebooks --force

Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in /content/drive/MyDrive/Dicoding/ProjectAnalisisSentimen/Script/Done/requirements.txt


# Preprocessing

In [ ]:
del clean_df

In [ ]:
!pip install nltk spacy

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd

# panggil dataset setelah scrapping
clean_df = pd.read_csv("/content/drive/MyDrive/Dicoding/ProjectAnalisisSentimen/Dataset/whatsapp_content_only.csv")
clean_df.head()

,content
0,When responding to messages through Android Au...
1,"I've been using this app for nearly a decade, ..."
2,Daily spam texts and calls after I got this ap...
3,The problem with perfection is it makes change...
4,Good interface and nice functionality. Does wh...


In [ ]:
import nltk
import string
import re
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy

# ================= Case Folding =====================
clean_df['content'] = clean_df['content'].str.lower()

# ===================== Cleaning ========================
def cleaning(text):
    # Remove tab, new line, and backslash
    text = text.replace('\\t', " ").replace('\\n', " ").replace('\\u', " ").replace('\\', "")
    # Remove non-ASCII characters
    text = text.encode('ascii', 'replace').decode('ascii')
    # Remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text).split())
    # Remove incomplete URLs
    text = text.replace("http://", " ").replace("https://", " ")
    # Remove numbers
    text = re.sub(r"\d+", "", text)
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove leading and trailing whitespace
    text = text.strip()
    # Replace multiple whitespace with a single space
    text = re.sub('\s+', ' ', text)
    # Remove single characters
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    return text  # Return the cleaned text

clean_df['text_cleaning'] = clean_df['content'].apply(cleaning)

# ====================== Tokenizing ================================
def word_tokenize_wrapper(text):
    return word_tokenize(text)

clean_df['text_tokens'] = clean_df['text_cleaning'].apply(word_tokenize_wrapper)

# ==================== Stopword Removal =========================

nltk.download('stopwords')

# mengimpor stopwords bahasa Inggris
list_stopwords = stopwords.words('english')

# fungsi untuk menghapus stopwords dari daftar kata
def stopwords_removal(words):
    return [word for word in words if word.lower() not in list_stopwords]

# menghapus stopwords pada kolom text_tokens
clean_df['text_stopwords'] = clean_df['text_tokens'].apply(stopwords_removal)

# ================= lemmatizing =====================

# muat spacy untuk lemmatizing
nlp = spacy.load('en_core_web_sm')

# fungsi lemmatizing pada teks
def spacy_lemmatize(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

# apply
clean_df['text_lemmatized'] = clean_df['text_stopwords'].apply(spacy_lemmatize)

clean_df = clean_df.drop_duplicates(subset=['text_lemmatized'])
clean_df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,content,text_cleaning,text_tokens,text_stopwords,text_lemmatized
0,when responding to messages through android au...,when responding to messages through android au...,"[when, responding, to, messages, through, andr...","[responding, messages, android, auto, acknowle...","[respond, message, android, auto, acknowledge,..."
1,"i've been using this app for nearly a decade, ...",ive been using this app for nearly decade and...,"[ive, been, using, this, app, for, nearly, dec...","[ive, using, app, nearly, decade, though, new,...","[I, ve, use, app, nearly, decade, though, new,..."
2,daily spam texts and calls after i got this ap...,daily spam texts and calls after got this app...,"[daily, spam, texts, and, calls, after, got, t...","[daily, spam, texts, calls, got, app, took, pu...","[daily, spam, text, call, get, app, take, put,..."
3,the problem with perfection is it makes change...,the problem with perfection is it makes change...,"[the, problem, with, perfection, is, it, makes...","[problem, perfection, makes, change, downgrade...","[problem, perfection, make, change, downgrade,..."
4,good interface and nice functionality. does wh...,good interface and nice functionality does wha...,"[good, interface, and, nice, functionality, do...","[good, interface, nice, functionality, need, w...","[good, interface, nice, functionality, need, w..."


kenapa menggunakan lemmatized?, karena jika menggunakan stemming, ada beberapa kata yang seharusnya tidak terkena efek stemming, kata itu malah berkurang hurufnya.

# Pelabelan

In [ ]:
!pip install textblob

In [ ]:
import pandas as pd
from textblob import TextBlob

# panggil dataset yang telah di preprocessing
df = clean_df[['text_lemmatized']]

df.head()

,text_lemmatized
0,"[respond, message, android, auto, acknowledge,..."
1,"[I, ve, use, app, nearly, decade, though, new,..."
2,"[daily, spam, text, call, get, app, take, put,..."
3,"[problem, perfection, make, change, downgrade,..."
4,"[good, interface, nice, functionality, need, w..."


In [ ]:
# fungsi untuk menggabungkan token menjadi kalimat
def combine_tokens(tokens):
    return ' '.join(tokens)

# apply fungsi yang dibuat
df['text_lemmatized'] = df['text_lemmatized'].apply(combine_tokens)

df.head()

<ipython-input-107-34c018f9f970>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_lemmatized'] = df['text_lemmatized'].apply(combine_tokens)


,text_lemmatized
0,respond message android auto acknowledge I ve ...
1,I ve use app nearly decade though new useful f...
2,daily spam text call get app take put two two ...
3,problem perfection make change downgrade do no...
4,good interface nice functionality need wish wo...


In [ ]:
# Fungsi untuk mendapatkan sentimen
def get_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# apply fungsi
df['label'] = df['text_lemmatized'].apply(get_sentiment)

# membuat DataFrame baru hanya dengan kolom text_lemmatized dan polarity
df = df[['text_lemmatized', 'label']]
df.head()

<ipython-input-108-27133234e900>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['text_lemmatized'].apply(get_sentiment)


,text_lemmatized,label
0,respond message android auto acknowledge I ve ...,negative
1,I ve use app nearly decade though new useful f...,positive
2,daily spam text call get app take put two two ...,positive
3,problem perfection make change downgrade do no...,positive
4,good interface nice functionality need wish wo...,positive


In [ ]:
# perhitungan jumlah pada setiap kelas label
label_counts = df['label'].value_counts()
print(label_counts)

label
positive    56775
negative    27261
neutral     13842
Name: count, dtype: int64


In [ ]:
# simpan ke csv
df.to_csv('/content/drive/MyDrive/Dicoding/ProjectAnalisisSentimen/Dataset/Whatsapp.csv')

# TF IDF SVM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# panggil dataset
df = pd.read_csv('/content/drive/MyDrive/Dicoding/ProjectAnalisisSentimen/Dataset/Whatsapp.csv')
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,"['respond', 'message', 'android', 'auto', 'ack...",negative
1,1,"['I', 've', 'use', 'app', 'nearly', 'decade', ...",positive
2,2,"['daily', 'spam', 'text', 'call', 'get', 'app'...",positive
3,3,"['problem', 'perfection', 'make', 'change', 'd...",positive
4,4,"['good', 'interface', 'nice', 'functionality',...",positive


In [ ]:
# Mengubah teks lemmatized ke format yang sesuai
df['text_lemmatized'] = df['text_lemmatized'].apply(lambda x: ' '.join(eval(x)))
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,respond message android auto acknowledge I ve ...,negative
1,1,I ve use app nearly decade though new useful f...,positive
2,2,daily spam text call get app take put two two ...,positive
3,3,problem perfection make change downgrade do no...,positive
4,4,good interface nice functionality need wish wo...,positive


In [ ]:
# mengubah label menjadi format numerik
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# membagi data
X = df['text_lemmatized']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(68514,) (29364,) (68514,) (29364,)


In [ ]:
# ekstraksi fitur dengan TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(X_train_tfidf.shape, X_test_tfidf.shape)

(68514, 36032) (29364, 36032)


In [ ]:
# melatih model SVM dengan kernel linear dan class weight 'balanced'
svm_model = SVC(kernel='linear', class_weight='balanced')
svm_model.fit(X_train_tfidf, y_train)

SVC(class_weight='balanced', kernel='linear')

In [ ]:
# melakukan prediksi dengan data latih
y_train_pred = svm_model.predict(X_train_tfidf)
print("Data Latih:")
print(classification_report(y_train, y_train_pred, target_names=label_encoder.classes_))

Data Latih:
              precision    recall  f1-score   support

    negative       0.91      0.96      0.93     19117
     neutral       0.90      0.98      0.94      9741
    positive       0.99      0.94      0.96     39656

    accuracy                           0.95     68514
   macro avg       0.93      0.96      0.95     68514
weighted avg       0.95      0.95      0.95     68514



In [ ]:
# melakukan prediksi dengan data uji
y_test_pred = svm_model.predict(X_test_tfidf)
print("Data Uji:")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

Data Uji:
              precision    recall  f1-score   support

    negative       0.87      0.93      0.90      8157
     neutral       0.86      0.96      0.91      4100
    positive       0.98      0.92      0.94     17107

    accuracy                           0.93     29364
   macro avg       0.90      0.94      0.92     29364
weighted avg       0.93      0.93      0.93     29364



In [ ]:
def predict_user_input(text, vectorizer, model, label_encoder):
    # ekstraksi fitur dengan TF-IDF
    text_tfidf = vectorizer.transform([text])

    # model melakukan prediksi
    prediction = model.predict(text_tfidf)

    # mengubah kelas prediksi menjadi label asli
    predicted_label = label_encoder.inverse_transform(prediction)
    return predicted_label[0]

# mencoba prediksi dengan ketikan dari user
user_input = input("Input Teks: ")
predicted_sentiment = predict_user_input(user_input, vectorizer, svm_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: great
teks ini masuk ke : positive


In [ ]:
# mencoba prediksi dengan ketikan dari user
user_input = input("Input Teks: ")
predicted_sentiment = predict_user_input(user_input, vectorizer, svm_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: bad
teks ini masuk ke : negative


In [ ]:
# mencoba prediksi dengan ketikan dari user
user_input = input("Input Teks: ")
predicted_sentiment = predict_user_input(user_input, vectorizer, svm_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: standart
teks ini masuk ke : neutral


 # GloVe Bidirectional LSTM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

# panggil dataset
df = pd.read_csv('/content/drive/MyDrive/Dicoding/ProjectAnalisisSentimen/Dataset/Whatsapp.csv')
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,"['respond', 'message', 'android', 'auto', 'ack...",negative
1,1,"['I', 've', 'use', 'app', 'nearly', 'decade', ...",positive
2,2,"['daily', 'spam', 'text', 'call', 'get', 'app'...",positive
3,3,"['problem', 'perfection', 'make', 'change', 'd...",positive
4,4,"['good', 'interface', 'nice', 'functionality',...",positive


In [ ]:
# menggabungkan token menjadi 1 kalimat
df['text_lemmatized'] = df['text_lemmatized'].apply(lambda x: ' '.join(eval(x)))
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,respond message android auto acknowledge I ve ...,negative
1,1,I ve use app nearly decade though new useful f...,positive
2,2,daily spam text call get app take put two two ...,positive
3,3,problem perfection make change downgrade do no...,positive
4,4,good interface nice functionality need wish wo...,positive


In [ ]:
# mengubah label menjadi format numerik
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,respond message android auto acknowledge I ve ...,0
1,1,I ve use app nearly decade though new useful f...,2
2,2,daily spam text call get app take put two two ...,2
3,3,problem perfection make change downgrade do no...,2
4,4,good interface nice functionality need wish wo...,2


In [ ]:
# membagi data
X = df['text_lemmatized']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(88090,) (9788,) (88090,) (9788,)


In [ ]:
# panggil file GloVe
glove_file = '/content/drive/MyDrive/Dicoding/ProjectAnalisisSentimen/Glove/glove.6B.100d.txt'

# membaca embedding GloVe
embeddings_index = {}
with open(glove_file, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f'Total {len(embeddings_index)} word vectors.')

Total 400000 word vectors.


In [ ]:
# menggunakan tokenizer untuk teks
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# mengonversi teks menjadi urutan indeks kata
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# padding sequences
maxlen = 184 # menyesuaikan maksimal panjang pada teks
X_train_padded = pad_sequences(X_train_sequences, maxlen=maxlen)
X_test_padded = pad_sequences(X_test_sequences, maxlen=maxlen)

# menyimpan informasi kata
word_index = tokenizer.word_index
print(f'ditemukan {len(word_index)} tokens yang unik / unique .')

ditemukan 42496 tokens yang unik / unique .


In [ ]:
# membuat matriks embedding
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(f'Embedding matrix shape: {embedding_matrix.shape}')


Embedding matrix shape: (42497, 100)


In [ ]:
# menghitung class weights untuk ketidakseimbangan kelas
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}

# mengubah label menjadi one-hot encoding
y_train_one_hot = to_categorical(y_train, num_classes=3)
y_test_one_hot = to_categorical(y_test, num_classes=3)

# membangun model LSTM
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False))
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=128)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))

# setup optimier, loss, dan metrics untuk mengukur akurasi
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 184, 100)          4249700   
                                                                 
 bidirectional (Bidirection  (None, 184, 256)          234496    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 184, 256)          0         
                                                                 
 batch_normalization (Batch  (None, 184, 256)          1024      
 Normalization)                                                  
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               394240    
 onal)                                                           
                                                        

In [ ]:
# membuat callbacks
checkpoint = ModelCheckpoint('best_lstm_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min')

# melatih model
model.fit(X_train_padded, y_train_one_hot, epochs=50, batch_size=32, validation_split=0.2,
          class_weight=class_weights_dict, callbacks=[checkpoint, early_stopping])

Epoch 1/50
2203/2203 [==============================] - 90s 37ms/step - loss: 0.6378 - accuracy: 0.7262 - val_loss: 0.3807 - val_accuracy: 0.8566
Epoch 2/50
2203/2203 [==============================] - 82s 37ms/step - loss: 0.3971 - accuracy: 0.8480 - val_loss: 0.3029 - val_accuracy: 0.8958
Epoch 3/50
2203/2203 [==============================] - 76s 35ms/step - loss: 0.3059 - accuracy: 0.8845 - val_loss: 0.3031 - val_accuracy: 0.8945
Epoch 4/50
2203/2203 [==============================] - 78s 35ms/step - loss: 0.2549 - accuracy: 0.9047 - val_loss: 0.1984 - val_accuracy: 0.9350
Epoch 5/50
2203/2203 [==============================] - 77s 35ms/step - loss: 0.2223 - accuracy: 0.9177 - val_loss: 0.1802 - val_accuracy: 0.9405
Epoch 6/50
2203/2203 [==============================] - 76s 35ms/step - loss: 0.1979 - accuracy: 0.9254 - val_loss: 0.1765 - val_accuracy: 0.9363
Epoch 7/50
2203/2203 [==============================] - 77s 35ms/step - loss: 0.1842 - accuracy: 0.9324 - val_loss: 0.1539 -

In [ ]:
# panggil model terbaik
best_model = load_model('best_lstm_model.keras')

# memprediksi dengan data latih
y_train_pred = best_model.predict(X_train_padded)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_classes = np.argmax(y_train_one_hot, axis=1)

print("Data Latih:")
print(classification_report(y_train_classes, y_train_pred_classes, target_names=label_encoder.classes_))

2753/2753 [==============================] - 35s 12ms/step
Data Latih:
              precision    recall  f1-score   support

    negative       0.94      0.95      0.95     24581
     neutral       0.95      0.98      0.96     12466
    positive       0.98      0.97      0.97     51043

    accuracy                           0.97     88090
   macro avg       0.96      0.97      0.96     88090
weighted avg       0.97      0.97      0.97     88090



In [ ]:
# memprediksi dengan data uji
y_test_pred = best_model.predict(X_test_padded)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_classes = np.argmax(y_test_one_hot, axis=1)

print("Data Uji:")
print(classification_report(y_test_classes, y_test_pred_classes, target_names=label_encoder.classes_))

306/306 [==============================] - 3s 11ms/step
Data Uji:
              precision    recall  f1-score   support

    negative       0.92      0.94      0.93      2693
     neutral       0.94      0.96      0.95      1375
    positive       0.97      0.96      0.97      5720

    accuracy                           0.96      9788
   macro avg       0.95      0.95      0.95      9788
weighted avg       0.96      0.96      0.96      9788



In [ ]:
def preprocess_and_predict(text, tokenizer, maxlen, model, label_encoder):
    # preprocessing teks yang diinput
    tokens = tokenizer.texts_to_sequences([text])
    tokens_padded = pad_sequences(tokens, maxlen=maxlen)

    # prediksi
    prediction = model.predict(tokens_padded)
    predicted_class = np.argmax(prediction, axis=1)

    # mengubah kelas prediksi menjadi label asli
    predicted_label = label_encoder.inverse_transform(predicted_class)
    return predicted_label[0]

# input dari user
user_input = input("Input Teks: ")
predicted_sentiment = preprocess_and_predict(user_input, tokenizer, maxlen, best_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: really worst
1/1 [==============================] - 0s 83ms/step
teks ini masuk ke : negative


In [ ]:
# input dari user
user_input = input("Input Teks: ")
predicted_sentiment = preprocess_and_predict(user_input, tokenizer, maxlen, best_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: great
1/1 [==============================] - 0s 19ms/step
teks ini masuk ke : positive


In [ ]:
# input dari user
user_input = input("Input Teks: ")
predicted_sentiment = preprocess_and_predict(user_input, tokenizer, maxlen, best_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: standart
1/1 [==============================] - 0s 27ms/step
teks ini masuk ke : neutral


# Word2Vec LSTM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import load_model
import gensim
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
# panggil dataset
df = pd.read_csv('/content/drive/MyDrive/Dicoding/ProjectAnalisisSentimen/Dataset/Whatsapp.csv')
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,"['respond', 'message', 'android', 'auto', 'ack...",negative
1,1,"['I', 've', 'use', 'app', 'nearly', 'decade', ...",positive
2,2,"['daily', 'spam', 'text', 'call', 'get', 'app'...",positive
3,3,"['problem', 'perfection', 'make', 'change', 'd...",positive
4,4,"['good', 'interface', 'nice', 'functionality',...",positive


In [ ]:
# menggabungkan token menjadi 1 kalimat
df['text_lemmatized'] = df['text_lemmatized'].apply(lambda x: ' '.join(eval(x)))
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,respond message android auto acknowledge I ve ...,negative
1,1,I ve use app nearly decade though new useful f...,positive
2,2,daily spam text call get app take put two two ...,positive
3,3,problem perfection make change downgrade do no...,positive
4,4,good interface nice functionality need wish wo...,positive


In [ ]:
# Mengubah label menjadi format numerik
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,respond message android auto acknowledge I ve ...,0
1,1,I ve use app nearly decade though new useful f...,2
2,2,daily spam text call get app take put two two ...,2
3,3,problem perfection make change downgrade do no...,2
4,4,good interface nice functionality need wish wo...,2


In [ ]:
# membagi data
X = df['text_lemmatized']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(88090,) (9788,) (88090,) (9788,)


In [ ]:
# menggunakan tokenizer untuk teks
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# mengonversi teks menjadi urutan indeks kata
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# padding sequences
maxlen = 184  # Sesuaikan dengan panjang maksimal teks Anda
X_train_padded = pad_sequences(X_train_sequences, maxlen=maxlen)
X_test_padded = pad_sequences(X_test_sequences, maxlen=maxlen)

# menyimpan informasi kata
word_index = tokenizer.word_index
print(f'ditemukan {len(word_index)} tokens yang unik / unique .')

ditemukan 42496 tokens yang unik / unique .


In [ ]:
# membuat daftar kata-kata dari data pelatihan
sentences = [sentence.split() for sentence in X_train]

# melatih model Word2Vec
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

print(f'Embedding matrix shape: {embedding_matrix.shape}')

Embedding matrix shape: (42497, 100)


In [ ]:
# latih model Word2Vec
sentences = [sentence.split() for sentence in X_train]
word2vec_model = Word2Vec(sentences, vector_size=embedding_dim, window=5, min_count=1, workers=4)

# memuat embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

# menghitung class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# mengubah label menjadi one-hot encoding
y_train_one_hot = to_categorical(y_train, num_classes=3)
y_test_one_hot = to_categorical(y_test, num_classes=3)

In [ ]:
# membangun model
model2 = Sequential()
model2.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False))
model2.add((LSTM(units=128, return_sequences=True)))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add((LSTM(units=128)))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add(Dense(3, activation='softmax'))
model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 184, 100)          4249700   
                                                                 
 lstm_3 (LSTM)               (None, 184, 128)          117248    
                                                                 
 dropout_3 (Dropout)         (None, 184, 128)          0         
                                                                 
 batch_normalization_3 (Bat  (None, 184, 128)          512       
 chNormalization)                                                
                                                                 
 lstm_4 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                      

In [ ]:
adam = Adam(learning_rate=0.001)
model2.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('best_lstm_model_2.keras', monitor='val_loss', save_best_only=True, mode='min')

# melatih model
model2.fit(X_train_padded, y_train_one_hot,
                    epochs=30,
                    batch_size=64,
                    validation_split=0.2,
                    class_weight=class_weights_dict,
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])

Epoch 1/30
1102/1102 [==============================] - 42s 34ms/step - loss: 0.6590 - accuracy: 0.7128 - val_loss: 0.5537 - val_accuracy: 0.7802 - lr: 0.0010
Epoch 2/30
1102/1102 [==============================] - 35s 32ms/step - loss: 0.4225 - accuracy: 0.8298 - val_loss: 0.3633 - val_accuracy: 0.8642 - lr: 0.0010
Epoch 3/30
1102/1102 [==============================] - 37s 34ms/step - loss: 0.3554 - accuracy: 0.8615 - val_loss: 0.3197 - val_accuracy: 0.8848 - lr: 0.0010
Epoch 4/30
1102/1102 [==============================] - 30s 27ms/step - loss: 0.3116 - accuracy: 0.8801 - val_loss: 0.3247 - val_accuracy: 0.8866 - lr: 0.0010
Epoch 5/30
1102/1102 [==============================] - 26s 24ms/step - loss: 0.2854 - accuracy: 0.8897 - val_loss: 0.5706 - val_accuracy: 0.7166 - lr: 0.0010
Epoch 6/30
1102/1102 [==============================] - 28s 25ms/step - loss: 0.2529 - accuracy: 0.9023 - val_loss: 0.2763 - val_accuracy: 0.9088 - lr: 5.0000e-04
Epoch 7/30
1102/1102 [====================

In [ ]:
# prediksi untuk data latih
y_train_pred_model2 = model2.predict(X_train_padded)
y_train_pred_classes_model2 = np.argmax(y_train_pred_model2, axis=1)
y_train_classes = np.argmax(y_train_one_hot, axis=1)

print("Data Latih")
print(classification_report(y_train_classes, y_train_pred_classes_model2))

2753/2753 [==============================] - 21s 8ms/step
Data Latih
              precision    recall  f1-score   support

           0       0.92      0.91      0.91     24581
           1       0.85      0.98      0.91     12466
           2       0.97      0.94      0.95     51043

    accuracy                           0.94     88090
   macro avg       0.91      0.94      0.92     88090
weighted avg       0.94      0.94      0.94     88090



In [ ]:
# prediksi untuk data uji
y_test_pred_model2 = model2.predict(X_test_padded)
y_test_pred_classes_model2 = np.argmax(y_test_pred_model2, axis=1)
y_test_classes = np.argmax(y_test_one_hot, axis=1)

print("Data Uji")
print(classification_report(y_test_classes, y_test_pred_classes_model2))

306/306 [==============================] - 2s 7ms/step
Data Uji
              precision    recall  f1-score   support

           0       0.89      0.88      0.89      2693
           1       0.83      0.96      0.89      1375
           2       0.96      0.93      0.94      5720

    accuracy                           0.92      9788
   macro avg       0.89      0.92      0.91      9788
weighted avg       0.92      0.92      0.92      9788



In [ ]:
best_model = load_model('best_lstm_model_2.keras')

def preprocess_and_predict(text, tokenizer, maxlen, best_model, label_encoder):
    # preprocessing teks yang diinput
    tokens = tokenizer.texts_to_sequences([text])
    tokens_padded = pad_sequences(tokens, maxlen=maxlen)

    # prediksi
    prediction = model.predict(tokens_padded)
    predicted_class = np.argmax(prediction, axis=1)

    # mengubah kelas prediksi menjadi label asli
    predicted_label = label_encoder.inverse_transform(predicted_class)
    return predicted_label[0]

# Input dari user
user_input = input("Input Teks: ")
predicted_sentiment = preprocess_and_predict(user_input, tokenizer, maxlen, best_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: worst
1/1 [==============================] - 1s 544ms/step
teks ini masuk ke : negative


In [ ]:
# Input dari user
user_input = input("Input Teks: ")
predicted_sentiment = preprocess_and_predict(user_input, tokenizer, maxlen, best_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: awesome
1/1 [==============================] - 0s 36ms/step
teks ini masuk ke : positive


In [ ]:
# Input dari user
user_input = input("Input Teks: ")
predicted_sentiment = preprocess_and_predict(user_input, tokenizer, maxlen, best_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: standart
1/1 [==============================] - 0s 151ms/step
teks ini masuk ke : neutral
